In [1]:
# All imports
from dotenv import load_dotenv
import os
from openai import OpenAI
from pathlib import Path

In [2]:
# Load API key from .env file
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

In [3]:
system_prompt = "You are a senior software architect specializing in microservices"

In [4]:
prompt = """
Given these requirements for a modular monolith:
    TEASTORE - Requirements

        1. WebUI Service
        - Functionality: Acts as the front-end interface for users, providing the user experience through Servlets and JSP files.
        - Responsibilities:
            - Retrieves images from the Image Provider.
            - Handles user interactions and displays product information.
            - Manages cookies for session and state management.
            - Provides a status page showing the state of the TeaStore.

        2. Image Provider Service
        - Functionality: Supplies images to the WebUI, encoding them as base64 strings for embedding in HTML.
        - Responsibilities:
            - Matches product IDs or UI names to unique image identifiers.
            - Handles image size requests, delivering the appropriate image or a standard "not found" image.
            - Implements an in-memory cache with LFU replacement strategy for efficient image delivery.

        3. Auth Service
        - Functionality: Manages user authentication and session management.
        - Responsibilities:
            - Handles user login and session validation.
            - Hashes passwords using BCrypt.
            - Validates session integrity using salted and hashed SessionBlobs stored in cookies.

        4. Persistence Provider Service
        - Functionality: Provides access to the relational database for data persistence.
        - Responsibilities:
            - Maps relational entities to JSON objects using EclipseLink JPA ORM.
            - Supports CRUD operations for persistent entities.
            - Acts as a caching layer with a second-level entity cache.

        5. Recommender Service
        - Functionality: Generates personalized product recommendations for users.
        - Responsibilities:
            - Trains on existing orders to provide recommendations based on user behavior.
            - Implements collaborative filtering algorithms (Slope One and nearest-neighbor approaches).
            - Adapts recommendations based on user history and current shopping cart.

        6. Registry Service
        - Functionality: Maintains information about the service instances in the TeaStore.
        - Responsibilities:
            - Tracks the number of online service instances and their locations.
            - Manages service registration and heartbeat signals to monitor service availability.
            - Facilitates easy configuration of multiple parallel TeaStores.

        7. Database (Relational Database Back-end)**
        - Functionality: Stores persistent data for the application.
        - Responsibilities:
            - Serves as the backend for the Persistence Provider, holding all necessary data for the application.


Generate a list of microservices in CSV format with the following columns:
    - Microservice
    - Responsibilities (separate multiple responsibilities with semicolons)
    - Communicates With (separate multiple services with semicolons)
"""

In [5]:
few_shot_example = """
Given these requirements for a modular monolith:
    SIMPLE_BOOKSTORE - Requirements

    1. StoreFront Service
    - Functionality: Provides the web interface for customers to browse books.
    - Responsibilities:
        - Renders HTML pages for the user.
        - Sends availability checks to the Inventory Service.
        - Manages user shopping cart session.

    2. Inventory Service
    - Functionality: Manages the logic regarding book stock and pricing.
    - Responsibilities:
        - Calculates final price including taxes.
        - Queries the Database to update stock levels.
        - Returns book details to the StoreFront.

    3. Database
    - Functionality: Central storage for the system.
    - Responsibilities:
        - Stores book metadata and inventory counts.

Generate a list of microservices in CSV format with the following columns:
- Microservice
- Responsibilities (separate multiple responsibilities with semicolons)
- Communicates With (separate multiple services with semicolons)
"""

In [6]:
example_response = """
Microservice, Responsibilities, Communicates With
StoreFront Service, Renders HTML pages for the user; Sends availability checks to the Inventory Service; Manages user shopping cart session, Inventory Service
Inventory Service, Calculates final price including taxes; Queries the Database to update stock levels; Returns book details to the StoreFront, Database
Database, Stores book metadata and inventory counts, None
"""

In [7]:
# ========================================================================
# Function: run_prompt
# Description: Executes a prompt using the configured model.
# Parameters:
#   - prompt: string containing the task description.
#   - model: OpenAI model to use (default: "o3").
# Returns:
#   - The content of the first response message, stripped of whitespace.
# =======================================================================
def run_prompt(messages, model="gpt-5"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    return response.choices[0].message.content.strip()

In [8]:
zero_rules = [
    {"role": "system", "content": f"{system_prompt}"},
    {"role": "user", "content": f"{prompt}"}
]

In [9]:
few_rules =[
    #system
    {"role": "system", "content": system_prompt},
    #example
    {"role": "user", "content": f"{few_shot_example}"},
    {"role": "assistant", "content": f"{example_response}"},
    #task
    {"role": "user", "content": f"{prompt}"}
]

## Project two: zero-shot

In [10]:

result = run_prompt(zero_rules)

output_dir = Path.home() / "JAW" / "Results" / "openai_results" / "Project Two" 
output_dir.mkdir(parents=True, exist_ok=True)

file_path = output_dir / f"zero-openai-project-two-results.csv"

with open(file_path, 'w', encoding='utf-8') as f:
    f.write(result)

print(f"Arquivo salvo com sucesso em: {file_path}")

Arquivo salvo com sucesso em: /home/renan-alves/JAW/Results/openai_results/Project Two/zero-openai-project-two-results.csv


## Project two: few-shot

In [11]:

result = run_prompt(few_rules)

output_dir = Path.home() / "JAW" / "Results" / "openai_results" / "Project Two" 
output_dir.mkdir(parents=True, exist_ok=True)

file_path = output_dir / f"few-openai-project-two-results.csv"

with open(file_path, 'w', encoding='utf-8') as f:
    f.write(result)

print(f"Arquivo salvo com sucesso em: {file_path}")

Arquivo salvo com sucesso em: /home/renan-alves/JAW/Results/openai_results/Project Two/few-openai-project-two-results.csv
